<a href="https://www.kaggle.com/code/davidhalim2004/data-cleaning?scriptVersionId=269113706" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Problem Formulation

## Problem Description:
This project aims to develop a machine learning model that predicts future sales
and demand by utilising historical sales data and external factors, including
product details, promotions, seasonality, holidays, and economic indicators. The
goal is to analyse historical patterns and generate reliable forecasts that help
businesses make data-driven decisions to reduce costs, increase efficiency, and
improve customer satisfaction by predicting the daily sales for the next 28 days.

## Objectives

● Collect and preprocess historical sales and demand data.

● Identify key features that influence sales trends.

● Build, train, and optimise forecasting models to predict future sales and
demand.

● Deploy the best-performing model to generate forecasts in real-time or in
batches.
## Data source:
Hierarchical sales data from Walmart, the world’s largest company by revenue in the US.



# Code setup

## Important libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## data reading

In [2]:
sales_validation=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
sales_validation.head(3)

In [3]:
cal=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
cal.head(3)

In [4]:
sell_price = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
sell_price.head(3)

In [4]:
data = pd.read_csv("/kaggle/input/depi-dataset/data.csv")

# Schema formating

## sales_validation file

In [6]:
data = sales_validation.melt(
    id_vars=["id","item_id", "dept_id","cat_id","store_id","state_id"],  # columns to keep
    var_name="d",                          # new column name for day labels (d_1, d_2, ...)
    value_name="sales"                     # new column name for sales values
)

'\ndata = sales_validation.melt(\n    id_vars=["id","item_id", "dept_id","cat_id","store_id","state_id"],  # columns to keep\n    var_name="d",                          # new column name for day labels (d_1, d_2, ...)\n    value_name="sales"                     # new column name for sales values\n)\n'

In [7]:
data.drop(columns=['id'],inplace=True)

## calendar file

In [8]:
cal["date"]= pd.to_datetime(cal["date"])

In [9]:

cal["event_name_1"]= cal["event_name_1"].fillna("No event")
cal["event_type_1"]= cal["event_type_1"].fillna("No event")
cal["event_name_2"]= cal["event_name_2"].fillna("No event")
cal["event_type_2"]= cal["event_type_2"].fillna("No event")

'\ncal["event_name_1"]= cal["event_name_1"].fillna("No event")\ncal["event_type_1"]= cal["event_type_1"].fillna("No event")\ncal["event_name_2"]= cal["event_name_2"].fillna("No event")\ncal["event_type_2"]= cal["event_type_2"].fillna("No event")\n'

In [10]:
# Merge data and cal dataframes on the 'd' column
merged_data = pd.merge(data, cal, on='d', how='left')

conditions = [
    merged_data["state_id"] == "CA",
    merged_data["state_id"] == "TX",
    merged_data["state_id"] == "WI"
]
choices= [
    merged_data["snap_CA"],
    merged_data["snap_TX"],
    merged_data["snap_WI"]
]
merged_data["snap"]= np.select(conditions, choices)

merged_data.drop(columns=['snap_CA','snap_TX','snap_WI'],inplace=True)

'\n# Merge data and cal dataframes on the \'d\' column\nmerged_data = pd.merge(data, cal, on=\'d\', how=\'left\')\n\nconditions = [\n    merged_data["state_id"] == "CA",\n    merged_data["state_id"] == "TX",\n    merged_data["state_id"] == "WI"\n]\nchoices= [\n    merged_data["snap_CA"],\n    merged_data["snap_TX"],\n    merged_data["snap_WI"]\n]\nmerged_data["snap"]= np.select(conditions, choices)\n\nmerged_data.drop(columns=[\'snap_CA\',\'snap_TX\',\'snap_WI\'],inplace=True)\n'

## sell_price file


In [11]:
final_data = temp_data.merge(
        sell_price,
        on=["store_id", "item_id", "wm_yr_wk"],
        how="left",
        validate="m:1"  # many sales rows per unique price row is expected
    )

'\nfinal_data = temp_data.merge(\n        sell_price,\n        on=["store_id", "item_id", "wm_yr_wk"],\n        how="left",\n        validate="m:1"  # many sales rows per unique price row is expected\n    )\n    '

In [12]:
final_data["price_in_dollars"]=final_data["sell_price"]*final_data["sales"]

In [13]:
final_data.to_csv("/kaggle/working/data.csv")

# EDA

In [14]:
data.isna().sum()

Unnamed: 0          0
item_id             0
store_id            0
d                   0
sales               0
date                0
wm_yr_wk            0
wday                0
event_name_1        0
event_type_1        0
event_name_2        0
event_type_2        0
snap                0
price_in_dollars    0
dtype: int64

In [15]:
print(data["event_name_1"].unique())
print(data["event_name_2"].unique())
print(data["event_type_1"].unique())
print(data["event_type_2"].unique())

['No event' "Mother's day" 'Halloween' 'VeteransDay' 'Thanksgiving'
 'Chanukah End' 'Christmas' 'NewYear' 'OrthodoxChristmas'
 'MartinLutherKingDay' 'SuperBowl' 'ValentinesDay' 'PresidentsDay'
 'LentStart' 'LentWeek2' 'Purim End' 'StPatricksDay' 'Easter' 'Pesach End'
 'Cinco De Mayo' 'MemorialDay' 'NBAFinalsStart' 'NBAFinalsEnd'
 'Ramadan starts' 'IndependenceDay' 'Eid al-Fitr' 'LaborDay' 'EidAlAdha'
 'ColumbusDay' "Father's day" 'OrthodoxEaster']


['No event' 'OrthodoxEaster' "Father's day" 'Cinco De Mayo' 'Easter']


['No event' 'Cultural' 'National' 'Religious' 'Sporting']


['No event' 'Religious' 'Cultural']


In [13]:
data.head(10)

,item_id,store_id,d,sales,date,wm_yr_wk,wday,event_name_1,event_type_1,event_name_2,event_type_2,snap,price_in_dollars
0,FOODS_1_001,CA_1,d_1,3,2011-01-29,11101,1,No event,No event,No event,No event,0,6.00
1,FOODS_1_001,CA_1,d_10,0,2011-02-07,11102,3,No event,No event,No event,No event,1,0.00
2,FOODS_1_001,CA_1,d_100,0,2011-05-08,11115,2,Mother's day,Cultural,No event,No event,1,0.00
3,FOODS_1_001,CA_1,d_1000,1,2013-10-24,11339,6,No event,No event,No event,No event,0,2.24
4,FOODS_1_001,CA_1,d_1001,0,2013-10-25,11339,7,No event,No event,No event,No event,0,0.00
5,FOODS_1_001,CA_1,d_1002,1,2013-10-26,11340,1,No event,No event,No event,No event,0,2.24
6,FOODS_1_001,CA_1,d_1003,0,2013-10-27,11340,2,No event,No event,No event,No event,0,0.00
7,FOODS_1_001,CA_1,d_1004,0,2013-10-28,11340,3,No event,No event,No event,No event,0,0.00
8,FOODS_1_001,CA_1,d_1005,0,2013-10-29,11340,4,No event,No event,No event,No event,0,0.00
9,FOODS_1_001,CA_1,d_1006,0,2013-10-30,11340,5,No event,No event,No event,No event,0,0.00


In [5]:
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 13 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   store_id          object 
 2   d                 object 
 3   sales             int64  
 4   date              object 
 5   wm_yr_wk          int64  
 6   wday              int64  
 7   event_name_1      object 
 8   event_type_1      object 
 9   event_name_2      object 
 10  event_type_2      object 
 11  snap              int64  
 12  price_in_dollars  float64
dtypes: float64(1), int64(4), object(8)
memory usage: 5.6+ GB
None


In [6]:
#Display summary statistics for all numerical columns
print(data.describe())

              sales      wm_yr_wk          wday          snap  \
count  5.832737e+07  5.832737e+07  5.832737e+07  5.832737e+07   
mean   1.126322e+00  1.133919e+04  3.997386e+00  3.293257e-01   
std    3.873108e+00  1.503742e+02  2.000652e+00  4.699684e-01   
min    0.000000e+00  1.110100e+04  1.000000e+00  0.000000e+00   
25%    0.000000e+00  1.121700e+04  2.000000e+00  0.000000e+00   
50%    0.000000e+00  1.133300e+04  4.000000e+00  0.000000e+00   
75%    1.000000e+00  1.144800e+04  6.000000e+00  1.000000e+00   
max    7.630000e+02  1.161300e+04  7.000000e+00  1.000000e+00   

       price_in_dollars  
count      5.832737e+07  
mean       3.217642e+00  
std        9.617942e+00  
min        0.000000e+00  
25%        0.000000e+00  
50%        0.000000e+00  
75%        2.980000e+00  
max        2.164320e+03  


In [7]:
#Count how many unique products and stores are in the dataset
print("Number of unique items:", data["item_id"].nunique())
print("Number of unique stores:", data["store_id"].nunique())


Number of unique items: 3049
Number of unique stores: 10


In [11]:
'''Check correlation between Sales column and Price_in_dollars column
A positive correlation means: when price increases, sales also increase'''
print(data[["sales", "price_in_dollars"]].corr())


                     sales  price_in_dollars
sales             1.000000          0.716545
price_in_dollars  0.716545          1.000000


In [14]:
'''Check which events appear most frequently in the dataset
This can help identify which holidays might affect sales'''

print("Most frequent event_name_1 values:")
print(data["event_name_1"].value_counts().head())
print("\nMost frequent event_type_1 values:")
print(data["event_type_1"].value_counts().head())


Most frequent event_name_1 values:
event_name_1
No event         53631910
LentStart          182940
LentWeek2          182940
ValentinesDay      182940
PresidentsDay      182940
Name: count, dtype: int64

Most frequent event_type_1 values:
event_type_1
No event     53631910
Religious     1585480
National      1554990
Cultural      1067150
Sporting       487840
Name: count, dtype: int64


# Data-Cleaning

In [17]:
data.drop(columns = "sell_price",inplace = True)
data.drop(columns="dept_id",inplace = True)
data.drop(columns = "cat_id",inplace = True)
data.drop(columns = "state_id",inplace = True)
data.drop(columns="month",inplace = True)
data.drop(columns="year",inplace = True)
data.drop(columns="weekday",inplace=True)

'\ndata.drop(columns = "sell_price",inplace = True)\ndata.drop(columns="dept_id",inplace = True)\ndata.drop(columns = "cat_id",inplace = True)\ndata.drop(columns = "state_id",inplace = True)\ndata.drop(columns="month",inplace = True)\ndata.drop(columns="year",inplace = True)\ndata.drop(columns="weekday",inplace=True)\n'

In [18]:
data.drop(columns="Unnamed: 0",inplace = True)

In [19]:
data.fillna(0,inplace = True)

In [20]:
data.sort_values(by=["store_id", "item_id", "d"], inplace=True)
data.reset_index(drop=True, inplace=True)

In [5]:
data.to_csv("/kaggle/working/data.csv",index= False)

NameError: name 'data' is not defined